# Scrabu Prototype (for a List of Shipment Numbers)

This notebook is for creating a prototype of the Scrabu project. The goal is to download DHL pages for a specific shipment number and scrap the shipment information from it.

#### Generate a list of shipment numbers

In [1]:
def generate_shipment_numbers(start_shipment_number=340434188193324407, size=100):
    import numpy as np
    shipment_numbers_list = []
    for i in np.arange(1, size):
        shipment_number = start_shipment_number + i
        shipment_numbers = list(map(int, str(shipment_number)))
        shipment_number = str(shipment_number).rjust(20, '0')
        shipment_numbers_list.append(shipment_number)
    return shipment_numbers_list

In [2]:
shipment_numbers = generate_shipment_numbers(start_shipment_number=340434188193324407, size=100)
print("First elements of the list", shipment_numbers[0:3])

First elements of the list ['00340434188193324408', '00340434188193324409', '00340434188193324410']


#### Download the HTML content for a specific shipment number

In [7]:
def request(shipment_number="340434188193324407", start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode="):
    import requests
    import numpy as np
    request_url = start_url + shipment_number
    return requests.get(request_url).content

In [8]:
html_response = request(shipment_number="340434188193324407")
print("Sample of HTML response:\n", html_response[0:100])

Sample of HTML response:
 b'\n<div id="verfolgen-main-content">\n  \n\n\n\n\n\n\t\n\n\n<script>\n  \n    window.__INITIAL_APP_STATE__ = {\n    '


#### Parsing HTML and converting it into JSON

In [9]:
def html_to_json(html, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=", shipment_number=None):
    from lxml import etree
    import json
    
    html_tree = etree.HTML(html)
    
    def clean_json(json):
        start = json.find('JSON.parse(')
        end = json.find('"),', start)
        json = json[start:end]
        json = json.replace('JSON.parse("', '')
        json = json.replace('\\', '')
        return json
    
    def find_json_element(html_tree):
        json_element = html_tree.xpath('//div')
        return str(etree.tostring(json_element[0]))
    
    dirty_json = find_json_element(html_tree)
    json_string = clean_json(dirty_json)
    return json.loads(json_string)

In [10]:
shipment_details_json = html_to_json(html_response)
print(shipment_details_json)

{'sendungen': [{'id': '00340434188193324407', 'hasCompleteDetails': True, 'sendungsinfo': {'gesuchteSendungsnummer': '340434188193324407', 'sendungsrichtung': 'ANKOMMEND'}, 'sendungsdetails': {'sendungsnummern': {'sendungsnummer': '00340434188193324407'}, 'eigenschaften': {}, 'sendungsverlauf': {'datumAktuellerStatus': '2019-10-10T15:05:00+02:00', 'aktuellerStatus': 'Die Sendung wurde erfolgreich zugestellt.', 'kurzStatus': 'Zustellung erfolgreich', 'fortschritt': 5, 'farbe': 0, 'events': [{'datum': '2019-10-07T16:27:00+02:00', 'status': 'Die Sendung wurde elektronisch angeku00FCndigt. Sobald die Sendung von uns bearbeitet wurde, erhalten Sie weitere Informationen.'}, {'datum': '2019-10-08T17:12:00+02:00', 'ort': 'Bremen', 'status': 'Die Sendung wurde von DHL bearbeitet und wird fu00FCr den Weitertransport in die Region des Empfu00E4ngers vorbereitet.'}, {'datum': '2019-10-09T10:25:00+02:00', 'ort': 'Neuwied', 'status': 'Die Sendung ist in der Region des Empfu00E4ngers angekommen und w

#### Structure the shipment details in JSON format

In [11]:
def shipment_details(shipment_details_json, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=", shipment_number="00340434188193324407"):
    import datetime
    delivery_history_dict = {}
    delivery_history_dict["shipment_number"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsnummern"].get("sendungsnummer")
    delivery_history_dict["crawltime"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    delivery_history_dict["url"] = start_url + shipment_number 
    delivery_history_dict["events"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsverlauf"].get("events", [])
    return delivery_history_dict

In [12]:
shipment_history = shipment_details(shipment_details_json)
print(shipment_history)

{'shipment_number': '00340434188193324407', 'crawltime': '2019-10-22 09:04:08', 'url': 'https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=00340434188193324407', 'events': [{'datum': '2019-10-07T16:27:00+02:00', 'status': 'Die Sendung wurde elektronisch angeku00FCndigt. Sobald die Sendung von uns bearbeitet wurde, erhalten Sie weitere Informationen.'}, {'datum': '2019-10-08T17:12:00+02:00', 'ort': 'Bremen', 'status': 'Die Sendung wurde von DHL bearbeitet und wird fu00FCr den Weitertransport in die Region des Empfu00E4ngers vorbereitet.'}, {'datum': '2019-10-09T10:25:00+02:00', 'ort': 'Neuwied', 'status': 'Die Sendung ist in der Region des Empfu00E4ngers angekommen und wird im nu00E4chsten Schritt zur Zustellbasis transportiert.'}, {'datum': '2019-10-10T05:58:00+02:00', 'ort': 'Bonn MechZB', 'status': 'Die Sendung wird fu00FCr die Verladung ins Zustellfahrzeug vorbereitet.'}, {'datum': '2019-10-10T09:19:00+02:00', 'status': 'Die Sendung wurde in das Zustellf

In [13]:
import json
def save_dictionary(shipment_history=None, filename=None):
    print("Writing file {}".format(filename))
    with open(filename, 'w') as f:
        json.dump(shipment_history, f)

In [14]:
save_dictionary(shipment_history, filename="../data/00340434188193324407.json")

Writing file ../data/00340434188193324407.json


#### Main function, which goes through all shipment numbers and uses the previous methods

In [15]:
import time
def main():
    shipment_numbers = generate_shipment_numbers(shipment_number=340434188193324407, size=100)
    for shipment_number in shipment_numbers:
        time.sleep(0.5)
        print("Processing shipment number {}".format(shipment_number))
        html_response = request(shipment_number=shipment_number)
        shipment_details_json = html_to_json(html_response)
        shipment_history = shipment_details(shipment_details_json)
        save_dictionary(shipment_history, filename="../data/{}.json".format(shipment_number))

In [16]:
main()

Processing shipment number 00340434188193324407
Writing file ../data/00340434188193324407.json
Processing shipment number 00340434188193324407
Writing file ../data/00340434188193324407.json
Processing shipment number 00340434188193324407
Writing file ../data/00340434188193324407.json
Processing shipment number 00340434188193324407
Writing file ../data/00340434188193324407.json
Processing shipment number 00340434188193324407
Writing file ../data/00340434188193324407.json
Processing shipment number 00340434188193324407
Writing file ../data/00340434188193324407.json
Processing shipment number 00340434188193324407
Writing file ../data/00340434188193324407.json
Processing shipment number 00340434188193324407
Writing file ../data/00340434188193324407.json
Processing shipment number 00340434188193324407
Writing file ../data/00340434188193324407.json
Processing shipment number 00340434188193324407
Writing file ../data/00340434188193324407.json
Processing shipment number 00340434188193324407
Wr

KeyboardInterrupt: 